# Random forest classification

## Single-node scikit-learn

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/1200px-Scikit_learn_logo_small.svg.png" width="300">

In [31]:
import os

MODEL_PATH = 'models'
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
numeric_feat = [
    'pickup_weekday', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'pickup_taxizone_id', 
    'dropoff_taxizone_id',
]
features = numeric_feat + categorical_feat
y_col = 'high_tip'

# Load data and feature engineering

Load a full month for this exercise

In [32]:
import os
import numpy as np
import datetime
import pandas as pd
import s3fs
import warnings
warnings.simplefilter("ignore")

import yaml
import snowflake.connector

creds = yaml.full_load(open('/home/jovyan/snowflake_creds.yml'))

conn = snowflake.connector.connect(
    warehouse='COMPUTE_WH',
    database='NYC_TAXI',
    schema='PUBLIC',
    **creds,
)

In [33]:
query = """
SELECT 
    pickup_taxizone_id,
    dropoff_taxizone_id,
    passenger_count,
    DIV0(tip_amount, fare_amount) > 0.2 AS high_tip,
    DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
    WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
    HOUR(pickup_datetime) AS pickup_hour,
    (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
    MINUTE(pickup_datetime) AS pickup_minute
FROM taxi_yellow
WHERE
    DATE_TRUNC('MONTH', pickup_datetime) = %s
"""
taxi = conn.cursor().execute(query, '2019-01-01')
columns = [x[0] for x in taxi.description]
taxi = pd.DataFrame(taxi.fetchall(), columns=columns)
taxi.columns = taxi.columns.str.lower()

In [34]:
print(f'Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB')

Num rows: 7667648, Size: 498.397248 MB


In [35]:
taxi_train = taxi[features + [y_col]]

In [36]:
taxi_train.high_tip.value_counts()

True     4001846
False    3665802
Name: high_tip, dtype: int64

In [37]:
taxi_train.head()

,pickup_weekday,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,pickup_taxizone_id,dropoff_taxizone_id,high_tip
0,3,6,78,48,2,100,132,False
1,3,6,78,56,1,236,236,False
2,3,6,78,21,1,162,234,True
3,3,6,78,33,1,186,161,False
4,3,6,78,52,1,162,170,True


# Train model

Setting `n_jobs=-1` tells scikit-learn to use all available cores on this machine to train models. Note that scikit-learn does NOT use the GPU, its using CPU cores here.

In [38]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

In [39]:
%%time
_ = rfc.fit(taxi_train[features], taxi_train[y_col])

CPU times: user 16min 27s, sys: 25 s, total: 16min 52s
Wall time: 4min 25s


## Save model

In [40]:
import cloudpickle

with open(f'{MODEL_PATH}/random_forest_scikit.pkl', 'wb') as f:
    cloudpickle.dump(rfc, f)

## Calculate metrics on test set

Use a different month for test set

In [41]:
taxi = conn.cursor().execute(query, '2019-02-01')
columns = [x[0] for x in taxi.description]
taxi = pd.DataFrame(taxi.fetchall(), columns=columns)
taxi.columns = taxi.columns.str.lower()
taxi_test = taxi

In [42]:
from sklearn.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[:, 1]
roc_auc_score(taxi_test[y_col], preds)

0.5716032156563667